# Loan Default Prediction

<img src="http://jihoon-kim.synology.me/wp-content/uploads/2017/07/LOAN-1024x512.jpg">

## Import Modules

In [2]:
# import modules
import pandas as pd

# Load Data

Data is provided by Kaggle Datasets(https://www.kaggle.com/wendykan/lending-club-loan-data).

### Data Description
>These files contain complete loan data for all loans issued through the 2007-2015, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. The file containing loan data through the "present" contains complete loan data for all loans issued through the previous completed calendar quarter. Additional features include credit scores, number of finance inquiries, address including zip codes, and state, and collections among others. The file is a matrix of about 890 thousand observations and 75 variables. A data dictionary is provided in a separate file.

In [4]:
# load data
loan = pd.read_csv('./data/loan.csv')

/home/jihoon_kim/anaconda3/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Exploratory Data Analysis

Let's quickly explore how this data looks like.

In [7]:
loan.dtypes

id                               int64
member_id                        int64
loan_amnt                      float64
funded_amnt                    float64
funded_amnt_inv                float64
term                            object
int_rate                       float64
installment                    float64
grade                           object
sub_grade                       object
emp_title                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
verification_status             object
issue_d                         object
loan_status                     object
pymnt_plan                      object
url                             object
desc                            object
purpose                         object
title                           object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs              

In [8]:
loan.isnull().sum()

id                                  0
member_id                           0
loan_amnt                           0
funded_amnt                         0
funded_amnt_inv                     0
term                                0
int_rate                            0
installment                         0
grade                               0
sub_grade                           0
emp_title                       51457
emp_length                          0
home_ownership                      0
annual_inc                          4
verification_status                 0
issue_d                             0
loan_status                         0
pymnt_plan                          0
url                                 0
desc                           761350
purpose                             0
title                             151
zip_code                            0
addr_state                          0
dti                                 0
delinq_2yrs                        29
earliest_cr_

There are some columns with huge number of null values! I will drop these columns.

In [9]:
null_cols = ['desc',
             'mths_since_last_delinq',
             'mths_since_last_record',
             'next_pymnt_d',
             'mths_since_last_major_derog',
             'annual_inc_joint',
             'dti_joint',
             'verification_status_joint',
             'open_acc_6m',
             'open_il_6m',
             'open_il_12m',
             'open_il_24m',
             'mths_since_rcnt_il',
             'total_bal_il',
             'il_util',
             'open_rv_12m',
             'open_rv_24m',
             'max_bal_bc',
             'all_util',
             'inq_fi',
             'total_cu_tl',
             'inq_last_12m'
             ]

In [10]:
def drop_null_columns(data):
    """Drop columns (most of values are null)"""
    data.drop(null_cols, axis=1, inplace=True)
    return None

In [11]:
drop_null_columns(loan)

# Modifying Target Column

The target column I am interested in is `loan_status`. Let's explore how this column is composed of.

In [12]:
loan.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

Meaning of loan status is described below:

|Loan Status|Meaning|
|---:|:---|
|Current|Loan is up to date on all outstanding payments.|
|Fully Paid|Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.|
|Charged Off|Loan for which there is no longer a reasonable expectation of further payments. Generally, Charge Off occurs no later than 30 days after the Default status is reached. Upon Charge Off, the remaining principal balance of the Note is deducted from the account balance.|
|Late (31-120 days)|Loan has not been current for 31 to 120 days.|
|Issued|New loan that has passed all LendingClub reviews, received full funding, and has been issued.|
|In Grace Period|Loan is past due but within the 15-day grace period.|
|Late (16-30 days)|Loan has not been current for 16 to 30 days.|
|Default|Loan has not been current for 121 days or more.|

First of all, `Current` and `Issued` loans are still in progress, so they cannot be used for training. Let's split them.

In [15]:
in_progress_index = ['Current', 'Issued']

In [16]:
def split_loan_in_progress(data):
    """Return table of loan in progress. It drops the loan in progress from loan data internally."""
    progress_bool = data.loan_status.isin(in_progress_index)
    loan_in_progress = data[progress_bool]
    data.drop(list(loan_in_progress.index), axis=0, inplace=True)
    return loan_in_progress

In [17]:
loan_in_progress = split_loan_in_progress(loan)

In [19]:
loan_in_progress

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
4,1075358,1311748,3000.0,3000.0,3000.000000,60 months,12.69,67.79,B,B5,...,Jan-2016,67.79,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
6,1069639,1304742,7000.0,7000.0,7000.000000,60 months,15.96,170.08,C,C5,...,Jan-2016,170.08,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
32,1065420,1299514,10000.0,10000.0,9975.000000,60 months,15.96,242.97,C,C5,...,Jan-2016,242.97,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
39,1069346,1304237,12500.0,12500.0,12475.000000,60 months,12.69,282.44,B,B5,...,Jan-2016,282.44,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
61,1068934,1277395,17500.0,8950.0,8925.000000,60 months,17.27,223.74,D,D3,...,Jan-2016,223.74,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
86,1063958,1297940,14000.0,14000.0,13975.000000,60 months,17.27,349.98,D,D3,...,Jan-2016,349.98,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
95,1068575,1303001,15300.0,15300.0,15275.000000,60 months,22.06,423.10,F,F4,...,Jan-2016,423.10,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
99,1067874,1302235,6000.0,6000.0,6000.000000,60 months,12.69,135.57,B,B5,...,Jan-2016,135.57,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
101,1034693,1264291,16000.0,16000.0,16000.000000,60 months,17.58,402.65,D,D4,...,Jan-2016,402.65,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
167,1067654,1302043,16000.0,16000.0,15975.000000,60 months,9.91,339.25,B,B1,...,Dec-2015,339.25,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN


In [20]:
loan

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,1077501,1296599,5000.0,5000.0,4975.000000,36 months,10.65,162.87,B,B2,...,Jan-2015,171.62,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.000000,60 months,15.27,59.83,C,C4,...,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.000000,36 months,15.96,84.33,C,C5,...,Jun-2014,649.91,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.000000,36 months,13.49,339.31,C,C1,...,Jan-2015,357.48,Jan-2015,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.000000,36 months,7.90,156.46,A,A4,...,Jan-2015,161.03,Sep-2015,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
7,1072053,1288686,3000.0,3000.0,3000.000000,36 months,18.64,109.43,E,E1,...,Jan-2015,111.34,Dec-2014,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
8,1071795,1306957,5600.0,5600.0,5600.000000,60 months,21.28,152.39,F,F2,...,Apr-2012,152.39,Aug-2012,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
9,1071570,1306721,5375.0,5375.0,5350.000000,60 months,12.69,121.45,B,B5,...,Nov-2012,121.45,Mar-2013,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
10,1070078,1305201,6500.0,6500.0,6500.000000,60 months,14.65,153.45,C,C3,...,Jun-2013,1655.54,Dec-2015,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
11,1069908,1305008,12000.0,12000.0,12000.000000,36 months,12.69,402.54,B,B5,...,Sep-2013,6315.30,Aug-2013,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
